### Import Required Libraries and Set Up Environment Variables

In [704]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')
#print(NASA_API_KEY)

### CME Data

In [705]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

#https://api.nasa.gov/DONKI/CME?startDate=yyyy-MM-dd&endDate=yyyy-MM-dd&api_key=DEMO_KEY
# Build URL for 
cme_url = f"{base_url}CME?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}" 

#cme_url


In [706]:
# Make a "GET" request for the CME URL and .json()store it in a variable named cme_response
cme_response = requests.get(cme_url)
cme_response

<Response [200]>

In [738]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()
#cme_json

In [708]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0],indent=4))

{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [709]:
# Convert cme_json to a Pandas DataFrame 

# Keep only the columns: activityID, startTime, linkedEvents
cme_df= pd.json_normalize(cme_json)
cme_df_selected = cme_df[["activityID","startTime","linkedEvents"]]
cme_df_selected



,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
...,...,...,...
5519,2024-05-01T06:36:00-CME-001,2024-05-01T06:36Z,None
5520,2024-05-01T11:36:00-CME-001,2024-05-01T11:36Z,None
5521,2024-05-01T12:36:00-CME-001,2024-05-01T12:36Z,None
5522,2024-05-01T17:36:00-CME-001,2024-05-01T17:36Z,None


In [710]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df_cleaned = cme_df_selected.dropna(subset=["linkedEvents"])
#cme_df_cleaned.reset_index
#print(cme_df_cleaned.index)
cme_df_cleaned

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...
...,...,...,...
5496,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,[{'activityID': '2024-04-26T00:17:00-IPS-001'}]
5499,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,[{'activityID': '2024-04-24T14:02:00-FLR-001'}]
5504,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,[{'activityID': '2024-04-25T17:03:00-FLR-001'}]
5516,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,[{'activityID': '2024-04-30T00:46:00-FLR-001'}]


In [711]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = [] 


# Iterate over each index in the DataFrame
for i in cme_df_cleaned.index:
    activityID = cme_df_cleaned.loc[i,"activityID"]
    startTime = cme_df_cleaned.loc[i,"startTime"]
    linkedevents = cme_df_cleaned.loc[i,"linkedEvents"]
    #print(linkedevents)
    # Iterate over each dictionary in the list
    for item in linkedevents:
        #expandedrow = {"activityID":activityID,"startTime":startTime,"linkedEvents":item["activityID"]}
        expandedrow = {"activityID":activityID,"startTime":startTime,"linkedEvents":item}
        expanded_rows.append(expandedrow)
        
    #expanded_rows
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
    #print(expanded_rows)
# Create a new DataFrame from the expanded rows
cmelinkedevents_df = pd.DataFrame(expanded_rows)
cmelinkedevents_df


,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}
...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'}
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'}
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'}
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'}


In [712]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
    try:
        inpdictactivityid = input_dict.get("activityID",None)
        return inpdictactivityid
    except(ValueError,TypeError) as e:
        #print("Key not found")
        return None

        # Log the error or print it for debugging

actid = extract_activityID_from_dict(cmelinkedevents_df.loc[21,"linkedEvents"])
print(actid)


2013-06-07T03:00:00-GST-001


In [713]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cmelinkedevents_df.loc[:,"GST_ActivityID"] = cmelinkedevents_df["linkedEvents"].apply(lambda events:extract_activityID_from_dict(events) )    
cmelinkedevents_df

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'},2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'},2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'},2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'},2024-04-30T00:46:00-FLR-001


In [ ]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:

cme_df_cleaned = cmelinkedevents_df[cmelinkedevents_df["GST_ActivityID"].str.contains("GST", case=False, na=False)]
cme_df_cleaned

,activityID,startTime,linkedEvents,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02T20:24Z,{'activityID': '2013-06-07T03:00:00-GST-001'},2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29T22:40Z,{'activityID': '2013-10-02T03:00:00-GST-001'},2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04T23:12Z,{'activityID': '2013-12-08T00:00:00-GST-001'},2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16T14:15Z,{'activityID': '2014-02-19T03:00:00-GST-001'},2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18T01:25Z,{'activityID': '2014-02-20T03:00:00-GST-001'},2014-02-20T03:00:00-GST-001
...,...,...,...,...
1519,2023-12-14T17:38:00-CME-001,2023-12-14T17:38Z,{'activityID': '2023-12-18T06:00:00-GST-001'},2023-12-18T06:00:00-GST-001
1645,2024-02-28T17:48:00-CME-001,2024-02-28T17:48Z,{'activityID': '2024-03-03T18:00:00-GST-001'},2024-03-03T18:00:00-GST-001
1665,2024-03-23T01:25:00-CME-001,2024-03-23T01:25Z,{'activityID': '2024-03-24T12:00:00-GST-001'},2024-03-24T12:00:00-GST-001
1669,2024-03-23T01:48:00-CME-001,2024-03-23T01:48Z,{'activityID': '2024-03-24T12:00:00-GST-001'},2024-03-24T12:00:00-GST-001


In [715]:
# print out the datatype of each column in this DataFrame:
cme_df_cleaned.info()
cme_df_cleaned

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 21 to 1689
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      61 non-null     object
 1   startTime       61 non-null     object
 2   linkedEvents    61 non-null     object
 3   GST_ActivityID  61 non-null     object
dtypes: object(4)
memory usage: 2.4+ KB


,activityID,startTime,linkedEvents,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02T20:24Z,{'activityID': '2013-06-07T03:00:00-GST-001'},2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29T22:40Z,{'activityID': '2013-10-02T03:00:00-GST-001'},2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04T23:12Z,{'activityID': '2013-12-08T00:00:00-GST-001'},2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16T14:15Z,{'activityID': '2014-02-19T03:00:00-GST-001'},2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18T01:25Z,{'activityID': '2014-02-20T03:00:00-GST-001'},2014-02-20T03:00:00-GST-001
...,...,...,...,...
1519,2023-12-14T17:38:00-CME-001,2023-12-14T17:38Z,{'activityID': '2023-12-18T06:00:00-GST-001'},2023-12-18T06:00:00-GST-001
1645,2024-02-28T17:48:00-CME-001,2024-02-28T17:48Z,{'activityID': '2024-03-03T18:00:00-GST-001'},2024-03-03T18:00:00-GST-001
1665,2024-03-23T01:25:00-CME-001,2024-03-23T01:25Z,{'activityID': '2024-03-24T12:00:00-GST-001'},2024-03-24T12:00:00-GST-001
1669,2024-03-23T01:48:00-CME-001,2024-03-23T01:48Z,{'activityID': '2024-03-24T12:00:00-GST-001'},2024-03-24T12:00:00-GST-001


In [736]:
# Convert the 'GST_ActivityID' column to string format 


#cme_df_cleaned["GST_ActivityID"] = cme_df_cleaned["GST_ActivityID"].apply(','.join)
cme_df_cleaned.loc[:,"GST_ActivityID"] = pd.Series(cme_df_cleaned["GST_ActivityID"],dtype="str")


# Convert startTime to datetime format  
cme_df_cleaned.loc[:,"startTime"] = pd.to_datetime(cme_df_cleaned["startTime"])
# Rename startTime to startTime_CME and activityID to cmeID

cme_df_renamed = cme_df_cleaned.rename(columns={"startTime":"startTime_CME","activityID":"cmeID"})

# Drop linkedEvents
cme_df_renamed = cme_df_renamed.drop("linkedEvents", axis=1)

# Verify that all steps were executed correctly
cme_df_renamed.dtypes

cmeID                          object
startTime_CME     datetime64[ns, UTC]
GST_ActivityID                 object
dtype: object

In [737]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme_df_renamed=cme_df_renamed[cme_df_renamed["GST_ActivityID"].str.contains("GST")]
cme_df_renamed

,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001
...,...,...,...
1519,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001
1645,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001
1665,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001
1669,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001


In [718]:
cme_df_renamed.dtypes

cmeID                          object
startTime_CME     datetime64[ns, UTC]
GST_ActivityID                 object
dtype: object

### GST Data

In [719]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"
gst_url

'https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2024-05-01&api_key=zkulxLj4KhX570kXSAAHgvRBBz5ZP7ysi0sul8ZS'

In [720]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)

In [ ]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
print(json.dumps(gst_json[:3],indent=4))



[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    },
    {
        "gstID": "2013-06-07T03:00:00-GST-001",
        "startTime": "2013-06-07T03:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-07T03:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/330/-1",
        "linkedEvents": [
            {
                "activityID": "2013-06-02T20:24:

In [722]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.json_normalize(gst_json)
gst_df_selected = gst_df[["gstID","startTime","linkedEvents"]]
gst_df_selected

# Keep only the columns: activityID, startTime, linkedEvents


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
...,...,...,...
112,2023-12-18T06:00:00-GST-001,2023-12-18T06:00Z,[{'activityID': '2023-12-14T17:38:00-CME-001'}...
113,2024-03-03T18:00:00-GST-001,2024-03-03T18:00Z,[{'activityID': '2024-02-28T17:48:00-CME-001'}...
114,2024-03-23T21:00:00-GST-001,2024-03-23T21:00Z,[{'activityID': '2024-03-23T12:49:00-IPS-001'}...
115,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,[{'activityID': '2024-03-23T01:25:00-CME-001'}...


In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df_cleaned = gst_df_selected.dropna(subset="linkedEvents")
gst_df_cleaned

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...
...,...,...,...
112,2023-12-18T06:00:00-GST-001,2023-12-18T06:00Z,[{'activityID': '2023-12-14T17:38:00-CME-001'}...
113,2024-03-03T18:00:00-GST-001,2024-03-03T18:00Z,[{'activityID': '2024-02-28T17:48:00-CME-001'}...
114,2024-03-23T21:00:00-GST-001,2024-03-23T21:00Z,[{'activityID': '2024-03-23T12:49:00-IPS-001'}...
115,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,[{'activityID': '2024-03-23T01:25:00-CME-001'}...


In [724]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
# Explode the linkedEvents column
gst_df_explode = gst_df_cleaned.explode("linkedEvents")
gst_df_explode.reset_index()
gst_df_explode.dropna(subset="linkedEvents")
gst_df_explode

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}
...,...,...,...
115,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-23T01:48:00-CME-001'}
115,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T14:10:00-IPS-001'}
115,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T16:25:00-MPC-001'}
116,2024-04-19T18:00:00-GST-001,2024-04-19T18:00Z,{'activityID': '2024-04-15T06:48:00-CME-001'}


In [725]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:

gst_df_explode.loc[:,"CME_ActivityID"] = gst_df_explode["linkedEvents"].apply(lambda events:extract_activityID_from_dict(events) )

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_filtered_df = gst_df_explode[gst_df_explode["CME_ActivityID"].str.contains("CME", case=False, na=False)]
gst_filtered_df





,gstID,startTime,linkedEvents,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,{'activityID': '2013-12-04T23:12:00-CME-001'},2013-12-04T23:12:00-CME-001
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,{'activityID': '2014-02-16T14:15:00-CME-001'},2014-02-16T14:15:00-CME-001
6,2014-02-20T03:00:00-GST-001,2014-02-20T03:00Z,{'activityID': '2014-02-18T01:25:00-CME-001'},2014-02-18T01:25:00-CME-001
...,...,...,...,...
112,2023-12-18T06:00:00-GST-001,2023-12-18T06:00Z,{'activityID': '2023-12-14T17:38:00-CME-001'},2023-12-14T17:38:00-CME-001
113,2024-03-03T18:00:00-GST-001,2024-03-03T18:00Z,{'activityID': '2024-02-28T17:48:00-CME-001'},2024-02-28T17:48:00-CME-001
115,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-23T01:25:00-CME-001'},2024-03-23T01:25:00-CME-001
115,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-23T01:48:00-CME-001'},2024-03-23T01:48:00-CME-001


In [726]:
# Convert the 'CME_ActivityID' column to string format 
gst_filtered_df = gst_filtered_df.copy()
gst_filtered_df["CME_ActivityID"] = gst_filtered_df["CME_ActivityID"].astype(str)
#gst_filtered_df.loc[:, "CME_ActivityID"] = gst_filtered_df["CME_ActivityID"].astype(str)

# # Convert startTime to datetime format  
gst_filtered_df["startTime"] = pd.to_datetime(gst_filtered_df["startTime"])
# # Rename startTime to startTime_GST 
gst_df_renamed = gst_filtered_df.rename(columns={"startTime":"startTime_GST"})
# # # Drop linkedEvents
gst_df_renamed = gst_df_renamed.drop("linkedEvents", axis=1)

# # # Verify that all steps were executed correctly


gst_df_renamed



,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
4,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
5,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
6,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001
...,...,...,...
112,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001
113,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001
115,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001
115,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001


In [727]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  

gst_df_renamed=gst_df_renamed[gst_df_renamed["CME_ActivityID"].str.contains("CME")]
gst_df_renamed.dtypes




gstID                          object
startTime_GST     datetime64[ns, UTC]
CME_ActivityID                 object
dtype: object

### Merge both datatsets

In [728]:
gst_df_renamed

,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
4,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
5,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
6,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001
...,...,...,...
112,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001
113,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001
115,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001
115,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001


In [729]:
cme_df_renamed

,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001
...,...,...,...
1519,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001
1645,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001
1665,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001
1669,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001


In [730]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
# Merge both DataFrames on the specified columns
merged_df = pd.merge(gst_df_renamed,cme_df_renamed,left_on=["gstID","CME_ActivityID"],right_on=["GST_ActivityID", "cmeID"],how="inner")
merged_df

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001
...,...,...,...,...,...,...
56,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001
57,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001
58,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001
59,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001


In [731]:
# Verify that the new DataFrame has the same number of rows as cme and gst

# Verify the resulting DataFrame has the expected number of rows
print("Number of rows in merged DataFrame:", len(merged_df))
print("Number of rows in cme_df_cleaned:", len(cme_df_renamed))
print("Number of rows in gst_df_cleaned:", len(gst_df_renamed))

merged_df

Number of rows in merged DataFrame: 61
Number of rows in cme_df_cleaned: 61
Number of rows in gst_df_cleaned: 61


,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001
...,...,...,...,...,...,...
56,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001
57,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001
58,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001
59,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001


### Computing the time it takes for a CME to cause a GST

In [732]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df["timeDiff"] = (merged_df["startTime_GST"]-merged_df["startTime_CME"]).dt.total_seconds()
merged_df

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,369360.0
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001,188400.0
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001,262080.0
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001,218700.0
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001,178500.0
...,...,...,...,...,...,...,...
56,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001,303720.0
57,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001,346320.0
58,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001,124500.0
59,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001,123120.0


In [733]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
merged_df.describe()

,timeDiff
count,61.000000
mean,257366.557377
std,85989.336914
min,117360.000000
25%,187200.000000
50%,244260.000000
75%,308100.000000
max,529200.000000


### Exporting data in csv format

In [734]:
# Export data to CSV without the index
merged_df.to_csv("export.csv")